In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time

In [ ]:
#load the balanced data
data_balanced = pd.read_csv('UNSW_NB15_balanced_smote.csv')

### BiLSTM Model for Multi-class Classification

In [15]:
# train an lstm model on the data
from keras.models import Sequential,load_model
from keras.layers import Dense, LSTM, Dropout, Activation, Bidirectional
from keras.callbacks import ModelCheckpoint
from sklearn import model_selection
from sklearn import metrics
import tensorflow as tf
X = data_balanced.drop(['attack_cat','label'], axis=1)
Y = data_balanced[['attack_cat']]
Y['attack_cat'] = Y['attack_cat'].astype('int')
# Y['label'] = Y['label'].astype('int')
# one hot encode attack_cat
Y = pd.get_dummies(Y,columns=['attack_cat'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
Y_train = np.asarray(Y_train).astype(np.int32)
X_test = np.asarray(X_test).astype(np.float32)
Y_test = np.asarray(Y_test).astype(np.int32)

print(X_train.shape)
# define the model
model = Sequential()
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)
model.add(Bidirectional(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='softmax',recurrent_activation='sigmoid')))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(64, return_sequences=True, activation='sigmoid')))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(64, activation='sigmoid')))
model.add(Dropout(0.1))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

mc = ModelCheckpoint('bilstm_best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
# train the model
start = time.time()
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True, callbacks=[mc])
end = time.time()
print("Time taken to train the model: ", end - start)

C:\Users\Ameya\AppData\Local\Temp\ipykernel_31992\3809319008.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['attack_cat'] = Y['attack_cat'].astype('int')


(446400, 1, 196)
Epoch 1/10
13948/13950 [============================>.] - ETA: 0s - loss: 0.5890 - accuracy: 0.7777
Epoch 1: val_accuracy improved from -inf to 0.81094, saving model to bilstm_best_model.h5
13950/13950 [==============================] - 150s 9ms/step - loss: 0.5889 - accuracy: 0.7777 - val_loss: 0.4932 - val_accuracy: 0.8109
Epoch 2/10


C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13949/13950 [============================>.] - ETA: 0s - loss: 0.4955 - accuracy: 0.8102
Epoch 2: val_accuracy improved from 0.81094 to 0.81868, saving model to bilstm_best_model.h5
13950/13950 [==============================] - 156s 11ms/step - loss: 0.4955 - accuracy: 0.8102 - val_loss: 0.4674 - val_accuracy: 0.8187
Epoch 3/10
13944/13950 [============================>.] - ETA: 0s - loss: 0.4778 - accuracy: 0.8155
Epoch 3: val_accuracy improved from 0.81868 to 0.82151, saving model to bilstm_best_model.h5
13950/13950 [==============================] - 141s 10ms/step - loss: 0.4778 - accuracy: 0.8156 - val_loss: 0.4596 - val_accuracy: 0.8215
Epoch 4/10
13943/13950 [============================>.] - ETA: 0s - loss: 0.4669 - accuracy: 0.8187
Epoch 4: val_accuracy improved from 0.82151 to 0.82280, saving model to bilstm_best_model.h5
13950/13950 [==============================] - 128s 9ms/step - loss: 0.4668 - accuracy: 0.8187 - val_loss: 0.4515 - val_accuracy: 0.8228
Epoch 5/10
13947/13

In [16]:
model = load_model('bilstm_best_model.h5')
# print model classifaciton report
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
# y_pred = np.argmax(y_pred, axis=1)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))


3488/3488 [==============================] - 17s 4ms/step
Classification Report
              precision    recall  f1-score   support

           0       0.70      0.75      0.73     18639
           1       0.88      0.61      0.72     18534
           2       0.76      0.84      0.80     18672
           3       1.00      0.97      0.99     18497
           4       0.99      0.73      0.84     18721
           5       0.93      0.82      0.87     18537

   micro avg       0.87      0.79      0.83    111600
   macro avg       0.88      0.79      0.82    111600
weighted avg       0.88      0.79      0.82    111600
 samples avg       0.79      0.79      0.79    111600

Accuracy:
0.7872401433691756


C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
# print latency for model prediction
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('Average Latency for model prediction:(ms)')
print((end-start)/len(y_pred)*1000)

3488/3488 [==============================] - 21s 6ms/step
Average Latency for model prediction:(ms)
0.22114267913244104
